# Sioux Falls example

In [1]:
import sys
sys.path.append('../..')

## File paths

In [2]:
fldr = 'D:/release/Sample models/sioux_falls_2020_02_15'

# remove the comments for the lines below to run the Chicago model example instead
# fldr = 'D:/release/Sample models/Chicago_2020_02_15'
# proj_name = 'chicagomodel.sqlite'

dt_fldr = '0_tntp_data'
prj_fldr = '1_project'
new_proj_folder = '1_project_temp'

# skm_fldr = '2_skim_results'
# assg_fldr = '4_assignment_results'
# dstr_fldr = '5_distribution_results'
# frcst_fldr = '6_forecast'
# ftr_fldr = '7_future_year_assignment'

## We copy the project to a different folder so we don't overwrite things like the matrix table

In [3]:
from shutil import copytree, rmtree
from os.path import isfile, isdir, join

if isdir(join(fldr, new_proj_folder)):
    rmtree(join(fldr, new_proj_folder))

copytree(join(fldr, prj_fldr), join(fldr, new_proj_folder))

'D:/release/Sample models/sioux_falls_2020_02_15\\1_project_temp'

## Opening the project

In [4]:
# Imports
from aequilibrae.project import Project
from os.path import join
from aequilibrae import logger
import logging

In [5]:
project = Project()
project.load(join(fldr, new_proj_folder))

#### We will make sure log comes to the terminal as well

In [6]:
import sys
# Because assignment takes a long time, we want the log to be shown here
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s;%(name)s;%(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

## Path computation

In [7]:
# imports
from aequilibrae.paths import PathResults, path_computation

In [8]:
# we build all graphs
project.network.build_graphs()
# We get warnings that several fields in the project are filled with NaNs.  Which is true, but we won't use those fields

2020-12-03 20:19:19,973;aequilibrae;WARNING ; Fields were removed from Graph for being non-numeric: modes,link_type
2020-12-03 20:19:19,979;aequilibrae;WARNING ; Field(s) name,lanes has(ve) at least one NaN value. Check your computations
2020-12-03 20:19:19,983;aequilibrae;WARNING ; Field(s) name,lanes has(ve) at least one NaN value. Check your computations
2020-12-03 20:19:19,986;aequilibrae;WARNING ; Field(s) name,lanes has(ve) at least one NaN value. Check your computations
2020-12-03 20:19:19,990;aequilibrae;WARNING ; Field(s) name,lanes has(ve) at least one NaN value. Check your computations


In [9]:
# we grab the graph for cars
graph = project.network.graphs['c']

# let's say we want to minimize distance
graph.set_graph('distance')

# And will skim time and distance while we are at it
graph.set_skimming(['free_flow_time', 'distance'])

# And we will allow paths to be compute going through other centroids/centroid connectors
# required for the Sioux Falls network, as all nodes are centroids
graph.set_blocked_centroid_flows(False)

# instantiate a path results object and prepare it to work with the graph
res = PathResults()
res.prepare(graph)

# compute a path from node 2 to 13
path_computation(2, 13, graph, res)

In [10]:
# We can get the sequence of nodes we traverse
res.path_nodes

array([ 2,  1,  3, 12, 13], dtype=int64)

In [11]:
# We can get the link sequence we traverse
res.path

array([ 3,  2,  7, 37], dtype=int64)

In [12]:
# We can get the mileposts for our sequence of nodes
res.milepost

array([ 0.,  6., 10., 14., 17.])

In [13]:
# If we want to compute the path for a different destination and same origin, we can just do this
# It is way faster when you have large networks
res.update_trace(4)

In [14]:
res.path_nodes

array([2, 6, 5, 4], dtype=int64)

## Skimming

In [15]:
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
from aequilibrae.paths import NetworkSkimming, SkimResults

In [16]:
# from before
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_blocked_centroid_flows(False)

In [17]:
# And run the skimming
skm = NetworkSkimming(graph)
skm.execute()

In [18]:
# The result is an AequilibraEMatrix object
skims = skm.results.skims

# Which we can manipute directly from its temp file, if we wish
skims.matrices

memmap([[[ 0.,  0.],
         [ 6.,  6.],
         [ 4.,  4.],
         ...,
         [20., 20.],
         [17., 17.],
         [15., 15.]],

        [[ 6.,  6.],
         [ 0.,  0.],
         [10., 10.],
         ...,
         [21., 21.],
         [23., 23.],
         [21., 21.]],

        [[ 4.,  4.],
         [10., 10.],
         [ 0.,  0.],
         ...,
         [16., 16.],
         [13., 13.],
         [11., 11.]],

        ...,

        [[20., 20.],
         [21., 21.],
         [16., 16.],
         ...,
         [ 0.,  0.],
         [ 4.,  4.],
         [ 5.,  5.]],

        [[17., 17.],
         [23., 23.],
         [13., 13.],
         ...,
         [ 4.,  4.],
         [ 0.,  0.],
         [ 2.,  2.]],

        [[15., 15.],
         [21., 21.],
         [11., 11.],
         ...,
         [ 5.,  5.],
         [ 2.,  2.],
         [ 0.,  0.]]])

In [19]:
# Ore access each matrix
skims.free_flow_time

memmap([[ 0.,  6.,  4.,  8., 10., 11., 16., 13., 15., 18., 14.,  8., 11.,
         18., 23., 18., 20., 18., 22., 22., 18., 20., 17., 15.],
        [ 6.,  0., 10., 11.,  9.,  5., 10.,  7., 14., 16., 17., 14., 17.,
         21., 19., 12., 14., 12., 16., 16., 22., 21., 23., 21.],
        [ 4., 10.,  0.,  4.,  6., 10., 15., 12., 11., 14., 10.,  4.,  7.,
         14., 19., 17., 19., 17., 21., 20., 14., 16., 13., 11.],
        [ 8., 11.,  4.,  0.,  2.,  6., 11.,  8.,  7., 10.,  6.,  8., 11.,
         10., 15., 13., 15., 13., 17., 17., 18., 18., 14., 15.],
        [10.,  9.,  6.,  2.,  0.,  4.,  9.,  6.,  5.,  8.,  8., 10., 13.,
         12., 14., 11., 13., 11., 15., 15., 19., 17., 16., 17.],
        [11.,  5., 10.,  6.,  4.,  0.,  5.,  2.,  9., 11., 12., 14., 17.,
         16., 14.,  7.,  9.,  7., 11., 11., 17., 16., 20., 20.],
        [16., 10., 15., 11.,  9.,  5.,  0.,  3., 12.,  9., 14., 19., 19.,
         17., 12.,  5.,  7.,  2.,  9.,  6., 12., 11., 15., 15.],
        [13.,  7., 12.,  8.

In [20]:
# We can save it to the project if we want
skm.save_to_project('base_skims')

# We can also retrieve this skim record to write something to its description
matrices = project.matrices
mat_record = matrices.get_record('base_skims')
mat_record.description = 'minimized FF travel time while also skimming distance'
mat_record.save()

2020-12-03 20:19:20,543;aequilibrae;WARNING ; Matrix Record has been saved to the database


# Traffic assignment with skimming

In [21]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass

In [22]:
# We use the exact same graph we had above

In [23]:
# We can get a matrix like this
demand_the_old_way = AequilibraeMatrix()
demand_the_old_way.load(join(fldr, new_proj_folder, 'matrices', 'demand.omx'))
demand_the_old_way.computational_view(['matrix']) # We will only assign one user class stored as 'matrix' inside the OMX file

# or directly from the project record
# so let's inspect what we have in the project
proj_matrices = project.matrices
proj_matrices.list()

,name,file_name,cores,procedure,procedure_id,timestamp,description,status
0,demand_aem,demand.aem,1,None,None,2020-11-24 08:46:42,Original data imported to AEM format\n,
1,demand_omx,demand.omx,1,None,None,2020-11-24 08:47:18,Original data imported to OMX format,
2,base_skims,base_skims.omx,2,Network skimming,5ed488a806404261baceb4ed86ff1637,2020-12-03 20:19:20.260987,minimized FF travel time while also skimming d...,


In [24]:
# Let's get it in this better way
demand = proj_matrices.get_matrix('demand_omx')
demand.computational_view(['matrix'])

In [25]:
assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)

# The first thing to do is to add at list of traffic classes to be assigned
assig.add_class(assigclass)


# We set these parameters only after adding one class to the assignment
assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.001

assig.execute() # we then execute the assignment

2020-12-03 20:19:21,422;aequilibrae;INFO ; bfw Assignment STATS
2020-12-03 20:19:21,424;aequilibrae;INFO ; Iteration, RelativeGap, stepsize
2020-12-03 20:19:21,544;aequilibrae;INFO ; 1,inf,1.0
2020-12-03 20:19:21,672;aequilibrae;INFO ; 2,0.8485503509703024,0.36497345609427145
2020-12-03 20:19:21,796;aequilibrae;INFO ; 3,0.3813926225800203,0.2298356924660528
2020-12-03 20:19:21,920;aequilibrae;INFO ; 4,0.19621277462606984,0.18591312145268074
2020-12-03 20:19:22,044;aequilibrae;INFO ; 5,0.09069073200924213,0.7090816523174254
2020-12-03 20:19:22,169;aequilibrae;INFO ; 6,0.20600048221061426,0.1229016022154401
2020-12-03 20:19:22,293;aequilibrae;INFO ; 7,0.06710568925282254,0.38638656717489844
2020-12-03 20:19:22,420;aequilibrae;INFO ; 8,0.10307514154369488,0.1093055036410267
2020-12-03 20:19:22,542;aequilibrae;INFO ; 9,0.04222147191362779,0.2487805192125393
2020-12-03 20:19:22,665;aequilibrae;INFO ; 10,0.05926435464772421,0.15904810628271004
2020-12-03 20:19:22,786;aequilibrae;INFO ; 11,0.

### Outputs

In [26]:
# Convergence report is easy to see
import pandas as pd
convergence_report = assig.report()
convergence_report.head()

,iteration,rgap,alpha,warnings,beta0,beta1,beta2
0,1,inf,1.000000,,1.000000,0.000000,0.000000
1,2,0.848550,0.364973,,1.000000,0.000000,0.000000
2,3,0.381393,0.229836,,1.000000,0.000000,0.000000
3,4,0.196213,0.185913,,0.959771,0.040229,0.000000
4,5,0.090691,0.709082,,0.687640,0.286705,0.025654


In [27]:
volumes = assig.results()
volumes.head()

,matrix_ab,matrix_ba,matrix_tot,Congested_Time_AB,Congested_Time_BA,Congested_Time_Max,Delay_factor_AB,Delay_factor_BA,Delay_factor_Max,VOC_AB,VOC_BA,VOC_max,PCE_AB,PCE_BA,PCE_tot
link_id,,,,,,,,,,,,,,,
1,4570.421761,NaN,4570.421761,6.0,NaN,6.0,1.0,NaN,1.0,0.176463,NaN,0.176463,4570.421761,NaN,4570.421761
2,8275.382482,NaN,8275.382482,4.0,NaN,4.0,1.0,NaN,1.0,0.353596,NaN,0.353596,8275.382482,NaN,8275.382482
3,4675.373252,NaN,4675.373252,6.0,NaN,6.0,1.0,NaN,1.0,0.180515,NaN,0.180515,4675.373252,NaN,4675.373252
4,5900.513362,NaN,5900.513362,5.0,NaN,5.0,1.0,NaN,1.0,1.190056,NaN,1.190056,5900.513362,NaN,5900.513362
5,8170.430991,NaN,8170.430991,4.0,NaN,4.0,1.0,NaN,1.0,0.349112,NaN,0.349112,8170.430991,NaN,8170.430991


In [28]:
# We could export it to CSV or AequilibraE data, but let's put it directly into the results database
assig.save_results('base_year_assignment')

In [29]:
# And save the skims
assig.save_skims('base_year_assignment_skims', which_ones='all', format='omx')

2020-12-03 20:19:27,413;aequilibrae;WARNING ; Matrix Record has been saved to the database


# Trip distribution

### Calibration

We will calibrate synthetic gravity models using the skims for TIME that we just generated

In [30]:
import numpy as np
from aequilibrae.distribution import GravityCalibration
from aequilibrae.matrix import AequilibraeMatrix

In [31]:
# Let's take another look at what we have in terms of matrices in the model
proj_matrices.list()

,name,file_name,cores,procedure,procedure_id,timestamp,description,status
0,demand_aem,demand.aem,1,None,None,2020-11-24 08:46:42,Original data imported to AEM format\n,
1,demand_omx,demand.omx,1,None,None,2020-11-24 08:47:18,Original data imported to OMX format,
2,base_skims,base_skims.omx,2,Network skimming,5ed488a806404261baceb4ed86ff1637,2020-12-03 20:19:20.260987,minimized FF travel time while also skimming d...,
3,base_year_assignment_skims,base_year_assignment_skims.omx,4,Traffic Assignment,0fdd7d3b1d7d41f1b4b785230dbf73b4,2020-12-03 20:19:20.883219,Skimming for assignment procedure,


In [32]:
# We need the demand
demand = proj_matrices.get_matrix('demand_aem')

# And the skims
imped = proj_matrices.get_matrix('base_year_assignment_skims')

In [33]:
# We can check which matrix cores were created for our skims to decide which one to use
imped.names

#Where free_flow_time_final is actually the congested time for the last iteration

['distance_blended',
 'distance_final',
 'free_flow_time_blended',
 'free_flow_time_final']

In [34]:
# But before using the data, let's get some impedance for the intrazonals
# Let's assume it is 75% of the closest zone

imped_core = 'free_flow_time_final'
imped.computational_view([imped_core])

# If we run the code below more than once, we will be overwriting the diagonal values with non-sensical data
# so let's zero it first
np.fill_diagonal(imped.matrix_view, 0)

# We compute it with a little bit of NumPy magic
intrazonals = np.amin(imped.matrix_view, where=imped.matrix_view>0, initial=imped.matrix_view.max(), axis=1)
intrazonals *= 0.75

# Then we fill in the impedance matrix
np.fill_diagonal(imped.matrix_view, intrazonals)


In [35]:
# Since we are working with an OMX file, we cannot overwrite a matrix on disk
# So we give a new name to save it
imped.save(names=['final_time_with_intrazonals'])

In [36]:
# This also updates these new matrices as those being used for computation
# As one can verify below
imped.view_names

['final_time_with_intrazonals']

In [37]:
# We set the matrices for being used in computation
demand.computational_view(['matrix'])

In [38]:
from math import log10, floor
def plot_tlfd(demand, skim, name):
    import matplotlib.pyplot as plt
    b = floor(log10(skim.shape[0]) * 10)
    n, bins, patches = plt.hist(np.nan_to_num(skim.flatten(),0), bins = b, weights=np.nan_to_num(demand.flatten()), density=False, facecolor='g', alpha=0.75)

    plt.xlabel('Trip length')
    plt.ylabel('Probability')
    plt.title('Trip-length frequency distribution')
    plt.savefig(name, format="png")
    plt.clf()

In [39]:
for function in ['power', 'expo']:
    model = GravityCalibration(matrix=demand, impedance=imped, function=function, nan_as_zero=True)
    model.calibrate()
    
    # we save the model
    model.model.save(join(fldr, new_proj_folder, f'{function}_model.mod'))
    
    # We save a trip length frequency distribution image
    plot_tlfd(model.result_matrix.matrix_view, imped.matrix_view, join(fldr, new_proj_folder, f'{function}_tfld.png') )
    
    # We can save the result of applying the model as well
    # we can also save the calibration report
    with open(join(fldr, new_proj_folder, f'{function}_convergence.log'), 'w') as otp:
        for r in  model.report:
            otp.write(r+'\n')

../..\aequilibrae\distribution\gravity_application.py:319: RuntimeWarning: divide by zero encountered in power
  * a)[:]
../..\aequilibrae\distribution\gravity_application.py:329: RuntimeWarning: invalid value encountered in multiply
  self.output.matrix_view[:, :] = self.output.matrix_view[:, :] * non_inf


<Figure size 432x288 with 0 Axes>

In [40]:
# We save a trip length frequency distribution image
plot_tlfd(demand.matrix_view, imped.matrix_view,join(fldr, new_proj_folder, 'demand_tfld.png') )

<Figure size 432x288 with 0 Axes>

# Forecast

* We create a set of *'future'* vectors using some random growth factors
* We apply the model for inverse power, as the TFLD seems to be a better fit for the actual one

In [41]:
from aequilibrae.distribution import Ipf, GravityApplication, SyntheticGravityModel, Ipf
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
import numpy as np

In [42]:
# We compute the vectors from our matrix
mat = AequilibraeMatrix()

mat.load(join(fldr, dt_fldr, 'demand.omx'))
mat.computational_view()
origins = np.sum(mat.matrix_view, axis=1)
destinations = np.sum(mat.matrix_view, axis=0)

args = {'file_path':join(fldr,  new_proj_folder, 'synthetic_future_vector.aed'),
        "entries": mat.zones, 
        "field_names": ["origins", "destinations"],
    "data_types": [np.float64, np.float64], 
        "memory_mode": False}

vectors = AequilibraeData()
vectors.create_empty(**args)

vectors.index[:] =mat.index[:]

# Then grow them with some random growth between 0 and 10% - Plus balance them
vectors.origins[:] = origins * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations[:] = destinations * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations *= vectors.origins.sum()/vectors.destinations.sum()

In [43]:
# Impedance 
imped = proj_matrices.get_matrix('base_year_assignment_skims')
imped.computational_view(['final_time_with_intrazonals'])

# If we wanted the main diagonal to not be considered...
# np.fill_diagonal(imped.matrix_view, np.nan)

In [44]:
for function in ['power', 'expo']:
    model = SyntheticGravityModel()
    model.load(join(fldr, new_proj_folder, f'{function}_model.mod'))

    outmatrix = join(proj_matrices.fldr, f'demand_{function}_model.aem') 
    apply = GravityApplication()
    args = {"impedance": imped,
            "rows": vectors,
            "row_field": "origins",
            "model": model,
            "columns": vectors,
            "column_field": "destinations",
            "nan_as_zero":True
            }

    gravity = GravityApplication(**args)
    gravity.apply()

    gravity.save_to_project(name=f'demand_{function}_model', file_name=f'demand_{function}_model.aem')
    
    # We get the output matrix and save it to OMX too,
    gravity.save_to_project(name=f'demand_{function}_model_omx', file_name=f'demand_{function}_model.omx')

2020-12-03 20:19:30,227;aequilibrae;WARNING ; Matrix Record has been saved to the database
2020-12-03 20:19:30,504;aequilibrae;WARNING ; Matrix Record has been saved to the database
2020-12-03 20:19:31,047;aequilibrae;WARNING ; Matrix Record has been saved to the database
2020-12-03 20:19:31,360;aequilibrae;WARNING ; Matrix Record has been saved to the database


In [45]:
# We update the matrices table/records and verify that the new matrices are indeed there
proj_matrices.update_database()
proj_matrices.list()

,name,file_name,cores,procedure,procedure_id,timestamp,description,status
0,demand_aem,demand.aem,1,None,None,2020-11-24 08:46:42,Original data imported to AEM format\n,
1,demand_omx,demand.omx,1,None,None,2020-11-24 08:47:18,Original data imported to OMX format,
2,base_skims,base_skims.omx,2,Network skimming,5ed488a806404261baceb4ed86ff1637,2020-12-03 20:19:20.260987,minimized FF travel time while also skimming d...,
3,base_year_assignment_skims,base_year_assignment_skims.omx,4,Traffic Assignment,0fdd7d3b1d7d41f1b4b785230dbf73b4,2020-12-03 20:19:20.883219,Skimming for assignment procedure,
4,demand_power_model,demand_power_model.aem,1,Synthetic trip distribution,1aaceb989bc7406580aff54d861862fa,2020-12-03 20:19:30.021451,Synthetic trip distribution. POWER,
5,demand_power_model_omx,demand_power_model.omx,1,Synthetic trip distribution,1aaceb989bc7406580aff54d861862fa,2020-12-03 20:19:30.021451,Synthetic trip distribution. POWER,
6,demand_expo_model,demand_expo_model.aem,1,Synthetic trip distribution,6e9b40919aec442e8475fb978a247e86,2020-12-03 20:19:30.849071,Synthetic trip distribution. EXPO,
7,demand_expo_model_omx,demand_expo_model.omx,1,Synthetic trip distribution,6e9b40919aec442e8475fb978a247e86,2020-12-03 20:19:30.849071,Synthetic trip distribution. EXPO,


### We now run IPF for the future vectors

In [46]:
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view()

args = {'matrix': demand,
        'rows': vectors,
        'columns': vectors,
        'column_field': "destinations",
        'row_field': "origins",
        'nan_as_zero': True}

ipf = Ipf(**args)
ipf.fit()


ipf.save_to_project(name='demand_ipf', file_name='demand_ipf.aem')
ipf.save_to_project(name='demand_ipf_omx', file_name='demand_ipf.omx')

2020-12-03 20:19:31,774;aequilibrae;WARNING ; Matrix Record has been saved to the database
2020-12-03 20:19:32,025;aequilibrae;WARNING ; Matrix Record has been saved to the database


In [47]:
proj_matrices.list()

,name,file_name,cores,procedure,procedure_id,timestamp,description,status
0,demand_aem,demand.aem,1,None,None,2020-11-24 08:46:42,Original data imported to AEM format\n,
1,demand_omx,demand.omx,1,None,None,2020-11-24 08:47:18,Original data imported to OMX format,
2,base_skims,base_skims.omx,2,Network skimming,5ed488a806404261baceb4ed86ff1637,2020-12-03 20:19:20.260987,minimized FF travel time while also skimming d...,
3,base_year_assignment_skims,base_year_assignment_skims.omx,4,Traffic Assignment,0fdd7d3b1d7d41f1b4b785230dbf73b4,2020-12-03 20:19:20.883219,Skimming for assignment procedure,
4,demand_power_model,demand_power_model.aem,1,Synthetic trip distribution,1aaceb989bc7406580aff54d861862fa,2020-12-03 20:19:30.021451,Synthetic trip distribution. POWER,
5,demand_power_model_omx,demand_power_model.omx,1,Synthetic trip distribution,1aaceb989bc7406580aff54d861862fa,2020-12-03 20:19:30.021451,Synthetic trip distribution. POWER,
6,demand_expo_model,demand_expo_model.aem,1,Synthetic trip distribution,6e9b40919aec442e8475fb978a247e86,2020-12-03 20:19:30.849071,Synthetic trip distribution. EXPO,
7,demand_expo_model_omx,demand_expo_model.omx,1,Synthetic trip distribution,6e9b40919aec442e8475fb978a247e86,2020-12-03 20:19:30.849071,Synthetic trip distribution. EXPO,
8,demand_ipf,demand_ipf.aem,1,Iterative Proportional fitting,56b2cf39dcdc408baa5980baf21fc36a,2020-12-03 20:19:31.592597,None,
9,demand_ipf_omx,demand_ipf.omx,1,Iterative Proportional fitting,56b2cf39dcdc408baa5980baf21fc36a,2020-12-03 20:19:31.592597,None,


# Future traffic assignment

In [48]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger
import logging

In [49]:
logger.info('\n\n\n TRAFFIC ASSIGNMENT FOR FUTURE YEAR')

2020-12-03 20:19:32,154;aequilibrae;INFO ; 


 TRAFFIC ASSIGNMENT FOR FUTURE YEAR


In [50]:
demand = proj_matrices.get_matrix('demand_power_model')

# let's see what is the core we ended up getting. It should be 'gravity'
demand.names

['gravity']

In [51]:
# Let's use the IPF matrix
demand.computational_view('gravity')

assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)

# The first thing to do is to add at list of traffic classes to be assigned
assig.add_class(assigclass)

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.001

assig.execute() # we then execute the assignment

2020-12-03 20:19:32,802;aequilibrae;INFO ; bfw Assignment STATS
2020-12-03 20:19:32,803;aequilibrae;INFO ; Iteration, RelativeGap, stepsize
2020-12-03 20:19:32,919;aequilibrae;INFO ; 1,inf,1.0
2020-12-03 20:19:33,045;aequilibrae;INFO ; 2,0.8712771889493577,0.30517056128802644
2020-12-03 20:19:33,173;aequilibrae;INFO ; 3,0.5503383654539055,0.1852809469254281
2020-12-03 20:19:33,298;aequilibrae;INFO ; 4,0.27029380895164345,0.20354817419136667
2020-12-03 20:19:33,419;aequilibrae;INFO ; 5,0.15396219080543105,0.47462987209525753
2020-12-03 20:19:33,550;aequilibrae;INFO ; 6,0.18912987666125858,0.15194208768256764
2020-12-03 20:19:33,668;aequilibrae;INFO ; 7,0.09634170923159564,0.33614706969234115
2020-12-03 20:19:33,792;aequilibrae;INFO ; 8,0.10208084520953109,0.9658737536794567
2020-12-03 20:19:33,923;aequilibrae;INFO ; 9,0.16759321645675332,0.11509190278139968
2020-12-03 20:19:34,045;aequilibrae;INFO ; 10,0.05735912832263536,0.14947048187646675
2020-12-03 20:19:34,175;aequilibrae;INFO ; 11

In [52]:
# We could export it to CSV or AequilibraE data, but let's put it directly into the results database
assig.save_results('future_year_assignment')

# And save the skims
assig.save_skims('future_year_assignment_skims', which_ones='all', format='omx')

2020-12-03 20:19:39,966;aequilibrae;WARNING ; Matrix Record has been saved to the database


## Close the project

In [53]:
project.close()

2020-12-03 20:19:40,057;aequilibrae;INFO ; Closed project on D:/release/Sample models/sioux_falls_2020_02_15\1_project_temp
